# Добро пожаловать в самостоятельный проект

Самостоятельный проект — это практическая проверка знаний, приобретённых вами на вводном курсе. Каждый раздел посвящён отдельной стадии анализа данных с экскурсом в основы Python. 

Проект выполняется в пять этапов:

•	Постановка задачи

•	Получение данных

•	Предобработка данных

•	Анализ данных

•	Оформление результатов

Для каждой части описаны шаги выполнения c теоретическим приложением. В Jupyter Notebook эти шаги связаны между собой выводами и результатами.

**Исследование: Музыка больших городов**

Яндекс.Музыка — это крупный продукт с огромным запасом данных для исследований. Команды таких сервисов для поддержания интереса к продукту и привлечения новых пользователей часто проводят исследования про пользователей. Чтобы удержать клиентов и привлечь новых, сделать бренд более узнаваемым, команда сервиса проводит исследования аудитории, и публикует интересные результаты. Например, интересно сравнить тексты, сочинённые нейросетью, с произведениями настоящих рэперов.
Есть исследование, которое напоминает наше: о музыкальных предпочтениях в разных городах России.
Итак, вопрос вам: как музыка, которая звучит по дороге на работу в понедельник утром, отличается от той, что играет в среду или в конце рабочей недели? Возьмите данные для Москвы и Петербурга. Сравните, что и в каком режиме слушают их жители.

План исследования

1.	Получение данных. Прочитайте данные, ознакомьтесь с ними.

2.	Предобработка данных. Избавьтесь от дубликатов, проблем с названиями столбцов и пропусками.

3.	Анализ данных. Ответьте на основные вопросы исследования, подготовьте отчётную таблицу или опишите полученный результат.

4.	Подведение итогов. Просмотрите выполненную работу и сформулируйте выводы.


# Этап 1. Получение данных

Изучим данные, предоставленные сервисом для проекта.

Прочитайте файл music_project.csv и сохраните его в переменной df.

Получите  первых 10 строк таблицы, а также общую информацию о данных таблицы df.

In [1]:
import pandas as pd
df = pd.read_csv('yandex_music.csv')
df.head(10)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Рассмотрим полученную информацию подробнее.
Всего в таблице 7 столбцов, тип данных у каждого столбца - строка.
Подробно разберём, какие в df столбцы и какую информацию они содержат:
•	userID — идентификатор пользователя;
•	Track — название трека;
•	artist — имя исполнителя;
•	genre — название жанра;
•	City — город, в котором происходило прослушивание;
•	time — время, в которое пользователь слушал трек;
•	Day — день недели.
Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.


## Выводы: 

Каждая строка таблицы содержит информацию о композициях определённого жанра в определённом исполнении, которые пользователи слушали в одном из городов в определённое время и день недели. Две проблемы, которые нужно решать: пропуски и некачественные названия столбцов. Для проверки рабочих гипотез особенно ценны столбцы (*вставьте сюда названия столбцов, наиболее существенных, на ва взгляд*. Данные из столбца *название столбца*  позволят узнать самые популярные жанры.

# Этап 2. Предобработка данных

Исключим пропуски, переименуем столбцы, а также проверим данные на наличие дубликатов.

Получаем перечень названий столбцов. Какая наблюдается проблема — кроме тех, что уже были названы ранее?


In [3]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

В названиях столбцов есть пробелы, которые могут затруднять доступ к данным.

Переименуем столбцы для удобства дальнейшей работы. Проверим результат.


In [5]:
new_names = {'Day': 'day',  '  City  ': 'city',  '  userID' : 'user_id', 'time': 'total_play_seconds', 'artist': 'artist_name', 'genre': 'genre_name', 'Track': 'track_name'}
df.rename(columns = new_names, inplace = True)
df.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city',
       'total_play_seconds', 'day'],
      dtype='object')

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [7]:
df.isnull().sum()

user_id                  0
track_name            1231
artist_name           7203
genre_name            1198
city                     0
total_play_seconds       0
day                      0
dtype: int64

Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные: скажем, не назван конкретный исполнитель народной песни. Хуже, если проблемы с записью данных. Каждый отдельный случай необходимо разобрать и выявить причину.

Заменяем пропущенные значения в столбцах с названием трека и исполнителя на строку 'unknown'. После этой операции нужно убедиться, что таблица больше не содержит пропусков.


In [9]:
df['track_name'] = df['track_name'].fillna('unknown')
df['artist_name'] = df['artist_name'].fillna('unknown')

In [11]:
df.isnull().sum()

user_id                  0
track_name               0
artist_name              0
genre_name            1198
city                     0
total_play_seconds       0
day                      0
dtype: int64

Удаляем в столбце с жанрами пустые значения; убеждаемся, что их больше не осталось.

In [13]:
df.dropna(subset = ['genre_name'], inplace = True)
df.isna().sum()

user_id               0
track_name            0
artist_name           0
genre_name            0
city                  0
total_play_seconds    0
day                   0
dtype: int64

Необходимо установить наличие дубликатов. Если найдутся, удаляем, и проверяем, все ли удалились.

In [15]:
df.duplicated().sum()

3755

In [17]:
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

Дубликаты могли появиться вследствие сбоя в записи данных. Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».

Сохраняем список уникальных значений столбца с жанрами в переменной genres_list.

Объявим функцию find_genre() для поиска неявных дубликатов в столбце с жанрами. Например, когда название одного и того же жанра написано разными словами.


In [19]:
genres_list = df['genre_name'].unique()

In [21]:
# <создание функции find_genre()>
# функция принимает как параметр строку с названием искомого жанра
# в теле объявляется переменная-счётчик, ей присваивается значение 0,
# затем цикл for проходит по списку уникальных значений
# если очередной элемент списка равен параметру функции, 
# то значение счётчика увеличивается на 1
# по окончании работы цикла функция возвращает значение счётчика
def find_genre(genre):
    count = 0
    for unique_genre in genres_list:
        if genre == unique_genre:
            count += 1
    return count

Вызов функции find_genre() для поиска различных вариантов названия жанра хип-хоп в таблице.

Правильное название — hiphop. Поищем другие варианты:

•	hip

•	hop

•	hip-hop


In [23]:
find_genre('hip')

1

In [25]:

find_genre('hop')

1

In [27]:
find_genre('hip-hop')

1

Объявим функцию find_hip_hop(), которая заменяет неправильное название этого жанра в столбце 'genre_name' на 'hiphop' и проверяет успешность выполнения замены.

Так исправляем все варианты написания, которые выявила проверка.


In [29]:
# <создание функции find_hip_hop()>
# функция принимает как параметры таблицу df и неверное название
# к столбцу 'genre_name' применяется специальный метод, 
# который заменяет второй параметр на строку 'hiphop'
# результат работы равен подсчитанному методом count() числу
#значений столбца, 
# которые равны второму параметру
# функция возвращает результат
def find_hip_hop(df, wrong_genre):
    count = 0
    for genre in df['genre_name']:
        if genre == wrong_genre:
            count += 1
    df['genre_name'] = df['genre_name'].replace(wrong_genre, 'hiphop')
    return count

In [37]:
# <замена одного неверного варианта на hiphop вызовом функции find_hip_hop()>
find_hip_hop(df, 'hip')
find_hip_hop(df, 'hop')
find_hip_hop(df, 'hip-hop')

1

Получаем общую информацию о данных. Убеждаемся, что чистка выполнена успешно.

In [39]:
genres_list = df['genre_name'].unique()
print(find_genre('hip'))
print(find_genre('hop'))
print(find_genre('hip-hop'))

0
0
0


## Вывод

На этапе предобработки в данных обнаружились не только пропуски и проблемы с названиями столбцов, но и всяческие виды дубликатов. Их удаление позволит провести анализ точнее. Поскольку сведения о жанрах важно сохранить для анализа, не просто удаляем все пропущенные значения, но заполним пропущенные имена исполнителей и названия треков. Имена столбцов теперь корректны и удобны для дальнейшей работы.


# Действительно ли музыку в разных городах слушают по-разному?

Была выдвинута гипотеза, что в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. Проверяем это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

Для каждого города устанавливаем количество прослушанных в эти дни композиций с известным жанром, и сравниваем результаты.
Группируем данные по городу и вызовом метода count() подсчитываем композиции, для которых известен жанр.


In [41]:
# <группировка данных таблицы df по столбцу 'city' и подсчёт количества значений столбца 'genre_name'>
df.groupby('city')['genre_name'].count()

city
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64

В *(вставьте название города)* прослушиваний больше, чем в *(вставьте название города)*, но это не значит, что *(вставьте название города)* более активна. У Яндекс.Музыки в целом больше пользователей в Москве, поэтому величины сопоставимы.
Сгруппируем данные по дню недели и подсчитаем прослушанные в понедельник, среду и пятницу композиции, для которых известен жанр.


In [43]:
# <группировка данных по столбцу 'weekday' и подсчёт количества значений столбца 'genre_name'>
df.groupby('day')['genre_name'].count()

day
Friday       21482
Monday       20866
Wednesday    17778
Name: genre_name, dtype: int64

Понедельник и пятница — время для музыки; по средам пользователи немного больше вовлечены в работу.

Создаём функцию number_tracks(), которая принимает как параметры таблицу, день недели и название города, а возвращает количество прослушанных композиций, для которых известен жанр. Проверяем количество прослушанных композиций для каждого города и понедельника, затем среды и пятницы.


In [45]:
# <создание функции number_tracks()>
# объявляется функция с тремя параметрами: df, day, city
# в переменной track_list сохраняются те строки таблицы df, для которых 
# значение в столбце 'weekday' равно параметру day
# и одновременно значение в столбце 'city' равно параметру city
# в переменной track_list_count сохраняется число значений столбца 'genre_name',
# рассчитанное методом count() для таблицы track_list
# функция возвращает значение track_list_count
def number_tracks(df, day, city):
    track_list = df[(df['day'] == day) & (df['city'] == city)]
    track_list_count = track_list['genre_name'].count()
    return track_list_count

In [47]:
# <список композиций для Москвы в понедельник>
number_tracks(df, 'Monday', 'Moscow')

15347

In [51]:
# <список композиций для Санкт-Петербурга в понедельник>
number_tracks(df, 'Monday', 'Saint-Petersburg')

5519

In [53]:
# <список композиций для Москвы в среду>
number_tracks(df, 'Wednesday', 'Moscow')

10865

In [55]:
# <список композиций для Санкт-Петербурга в среду>
number_tracks(df, 'Wednesday', 'Saint-Petersburg')

6913

In [57]:
# <список композиций для Москвы в пятницу>
number_tracks(df, 'Friday', 'Moscow')

15680

In [59]:
# <список композиций для Санкт-Петербурга в пятницу>
number_tracks(df, 'Friday', 'Saint-Petersburg')

5802

Сведём полученную информацию в одну таблицу, где ['city', 'monday', 'wednesday', 'friday'] названия столбцов.

In [61]:
# <таблица с полученными данными>
cities = ['Moscow', 'Saint-Petersburg']
monday_count = []
wednesday_count = []
friday_count = []

for city in cities:
    monday_count.append(number_tracks(df, 'Monday', city))
    wednesday_count.append(number_tracks(df, 'Wednesday', city))
    friday_count.append(number_tracks(df, 'Friday', city))

data = {'city': cities, 'monday': monday_count, 'wednesday': wednesday_count, 'friday': friday_count}
summary_df = pd.DataFrame(data)

summary_df

,city,monday,wednesday,friday
0,Moscow,15347,10865,15680
1,Saint-Petersburg,5519,6913,5802


## Вывод

*Запишите здесь ваши выводы.Опишите закономерности, которые вы наблюдаете по дням недели и городам*

# Утро понедельника и вечер пятницы — разная музыка или одна и та же?

Ищем ответ на вопрос, какие жанры преобладают в разных городах в понедельник утром и в пятницу вечером. Есть предположение, что в понедельник утром пользователи слушают больше бодрящей музыки (например, жанра поп), а вечером пятницы — больше танцевальных (например, электронику).
Получим таблицы данных по Москве moscow_general и по Санкт-Петербургу spb_general.


In [63]:
moscow_general = df[ df['city'] == 'Moscow']
moscow_general.head(10)
spb_general = df[ df['city'] == 'Saint-Petersburg']
spb_general.head(10)

,user_id,track_name,artist_name,genre_name,city,total_play_seconds,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday
14,94EB25C2,Make Love Whenever You Can,Arabesque,pop,Saint-Petersburg,13:22:08,Wednesday
15,81D05C7D,SLAVES OF FEAR,HEALTH,alternative,Saint-Petersburg,20:54:48,Monday
16,39DE290E,Веснянка,Веданъ Колодъ,folk,Saint-Petersburg,14:36:47,Monday
17,58AE138A,Pat-a-Cake,Mother Goose Club,children,Saint-Petersburg,20:04:56,Monday
21,B8E29C9C,Самота,unknown,folk,Saint-Petersburg,20:20:49,Wednesday


Создаём функцию genre_weekday(), которая возвращает список жанров по запрошенному дню недели и времени суток с такого-то часа по такой-то.

In [65]:
# объявление функции genre_weekday() с параметрами df, day, time1, time2
# в переменной genre_list сохраняются те строки df, для которых одновременно:
# 1) значение в столбце 'weekday' равно параметру day,
# 2) значение в столбце 'time' больше time1 и
# 3) меньше time2.
# в переменной genre_list_sorted сохраняются в порядке убывания  
# первые 10 значений Series, полученной подсчётом числа значений 'genre_name'
# сгруппированной по столбцу 'genre_name' таблицы genre_list
# функция возвращает значение genre_list_sorted
def genre_weekday(df, day, time1, time2):
    genre_list = df[ (df['day'] == day) & (df['total_play_seconds'] > time1) & (df['total_play_seconds'] < time2) ]
    genre_list_sorted = genre_list.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)
    genre_list_sorted = genre_list_sorted.head(10)
    return genre_list_sorted

Cравниваем полученные результаты по таблице для Москвы и Санкт-Петербурга в понедельник утром (с 7 до 11) и в пятницу вечером (с 17 до 23).

In [67]:
# <вызов функции для утра понедельника в Москве (вместо df таблица moscow_general)>
genre_weekday(moscow_general, 'Monday', '07:00:00', '11:00:00')

genre_name
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
classical      157
Name: genre_name, dtype: int64

In [69]:
# <вызов функции для утра понедельника в Петербурге (вместо df таблица spb_general)>
genre_weekday(spb_general, 'Monday', '07:00:00', '11:00:00')

genre_name
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre_name, dtype: int64

In [71]:
# <вызов функции для вечера пятницы в Москве>
genre_weekday(moscow_general, 'Monday', '17:00:00', '23:00:00')

genre_name
pop            717
dance          524
rock           518
electronic     485
hiphop         238
alternative    182
world          172
classical      172
ruspop         149
rusrap         133
Name: genre_name, dtype: int64

In [73]:
# <вызов функции для вечера пятницы в Питере>
genre_weekday(spb_general, 'Monday', '17:00:00', '23:00:00')

genre_name
pop            263
rock           208
electronic     192
dance          191
hiphop         104
alternative     72
classical       71
jazz            57
rusrap          54
ruspop          53
Name: genre_name, dtype: int64

Популярные жанры в понедельник утром в Питере и Москве оказались похожи: везде, как и предполагалось, популярен *(вставьте сюда название жанра)*. Несмотря на это, концовка топ-10 для двух городов различается: в Питере в топ-10 входит джаз и русский рэп, а в Москве жанр world.

*(Опишите здесь, меняется ли ситуация в музыкальными предпочтениями в конце недели, заметна ли разница в концовке ТОП-10)*

## Вывод
Жанр *(вставьте сюда название жанра)* безусловный лидер, а топ-5 в целом не различается в обеих столицах. При этом видно, что концовка списка более «живая»: для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени.


# Москва и Питер — две разные столицы, два разных направления в музыке. Правда?

Гипотеза: Питер богат своей рэп-культурой, поэтому это направление там слушают чаще, а Москва — город контрастов, но основная масса пользователей слушает попсу.

Сгруппируем таблицу moscow_general по жанру, сосчитаем численность композиций каждого жанра методом count(), отсортируем в порядке убывания и сохраним результат в таблице moscow_genres.

Просмотрим первые 10 строк этой новой таблицы.


In [75]:
# одной строкой: группировка таблицы moscow_general по столбцу 'genre_name', 
# подсчёт числа значений 'genre_name' в этой группировке методом count(), 
# сортировка Series в порядке убывания и сохранение в moscow_genres 
moscow_genres =  moscow_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)

In [77]:
# <просмотр первых 10 строк moscow_genres>
moscow_genres.head(10)

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64

Сгруппируем таблицу spb_general по жанру, сосчитаем численность композиций каждого жанра методом count(), отсортируем в порядке убывания и сохраним результат в таблице spb_genres.

Просматриваем первые 10 строк этой таблицы. Теперь можно сравнивать два города.


In [79]:
# <группировка таблицы spb_general, расчёт, сохранение в spb_genres>
spb_genres =  spb_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)

In [81]:
# <просмотр первых 10 строк spb_genres>
spb_genres.head(10)

genre_name
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre_name, dtype: int64

## Вывод
В Москве, кроме абсолютно популярного жанра поп, есть направление русской популярной музыки. Значит, что интерес к этому жанру шире. А рэп, вопреки предположению, занимает в обоих городах близкие позиции.


# Результаты исследования

Рабочие гипотезы:

•	музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;

•	списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;

•	население двух городов предпочитает разные музыкальные жанры.

**Общие результаты**

Москва и Петербург сходятся во вкусах: везде преобладает популярная музыка. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Но между городами в разрезе дней неделей наблюдается зеркальность относительно среды: Москва больше слушает в понедельник и пятницу, а Петербург наоборот - больше в среду, но меньше в понедельник и пятницу.
В результате первая гипотеза *подтверждена / не подтверждена*, вторая гипотеза *подтверждена / не подтверждена* и третья не подтверждена >.*подтверждена / не подтверждена*
